In [1]:
import pandas as pd 
import numpy as np 
import plotly.express as px 
import petpy 
import pickle
import sqlite3
import time
import datetime
import urllib
import requests

## Functions  

### Get Pagination Details

In [2]:
def create_header(creds):
    endpoint = 'oauth2/token'
    host = 'http://api.petfinder.com/v2/'
    url = urllib.parse.urljoin(host, endpoint)

    data = {
        'grant_type': 'client_credentials',
        'client_id': creds['key'],
        'client_secret': creds['secret']
    }

    r = requests.post(url, data=data)
    api_key = r.json()['access_token']
    headers = {'Authorization' : 'Bearer ' + api_key}
    return(headers)

In [3]:
def get_pagination_details(params, header):
    animals_endpt = 'https://api.petfinder.com/v2/animals/'
    results = requests.get(animals_endpt
                       , params=params
                       , headers=headers)
    assert(results.status_code == 200), f"Status code not 200: It's {results.status_code}"
    results_json = results.json()
    keeper_results = {}
    keeper_results['n_pages'] = results_json['pagination']['total_pages']
    keeper_results['total_count'] = results_json['pagination']['total_count']
    return(keeper_results)

### Create Petpy dict  

In [4]:
def create_petpy_dict(params, pages):
    petpy_dict = params.copy()
    petpy_dict['animal_type'] = petpy_dict.pop('type')
    petpy_dict['results_per_page'] = petpy_dict.pop('limit')
    petpy_dict['pages'] = pages
    petpy_dict['return_df'] = True
    del(petpy_dict['page'])
    if 'type' in petpy_dict.keys():
        del(petpy_dict['type'])
    print(petpy_dict)
    return(petpy_dict)

### Export to SQL

In [5]:
def _prepare_df_for_export(adoptable_pets):
    # Prep dataset for export into SQL 
    adoptable_pets = adoptable_pets.loc[:,~adoptable_pets.columns.duplicated()]    
    adoptable_pets['date_saved'] = str(datetime.datetime.now().date())
    for x in ['tags', 'photos', 'videos']:
        if x in adoptable_pets.columns:
            adoptable_pets[x] = str(adoptable_pets[x])
    return(adoptable_pets)

In [6]:
def _create_con(path):
    con = sqlite3.connect(path)
    return(con)

In [7]:
def _create_tbl_name(petpy_dict):
    tbl_name = f"{petpy_dict['animal_type']}_{petpy_dict['location']}_{petpy_dict['distance']}mi"
    return(tbl_name)

In [8]:
def append_df(adoptable_pets, petpy_dict, path= '/home/malcolm/petfinder/data/petfinder.db'):
    adoptable_pets = _prepare_df_for_export(adoptable_pets)
    con = _create_con(path)
    tbl_name = _create_tbl_name(petpy_dict)
    adoptable_pets.to_sql(tbl_name, con, index=False, if_exists='append')
    print(adoptable_pets.shape[0], " pets added") 
    con.commit()
    con.close()

## Running it

In [9]:
with open('/home/malcolm/credentials/petfinder_credentials.pkl', 'rb') as hnd:
    creds = pickle.load(hnd)

In [10]:
dog_params = dict(type='dog', 
    location='10475', distance=30,
    limit=100, page=1)
cat_params = dict(type='cat', 
    location='10475', distance=30,
    limit=100, page=1)

In [12]:
headers = create_header(creds)
dog_pagination = get_pagination_details(dog_params, headers)
dog_pages = dog_pagination['n_pages']
cat_pagination = get_pagination_details(cat_params, headers)
cat_pages = cat_pagination['n_pages']


In [13]:
pf = petpy.Petfinder(key=creds['key'], secret=creds['secret'])

dogs_petpy_dict = create_petpy_dict(dog_params, dog_pages)
adoptable_dogs = pf.animals(**dogs_petpy_dict)
append_df(adoptable_dogs, dogs_petpy_dict)

{'location': '10475', 'distance': 30, 'animal_type': 'dog', 'results_per_page': 100, 'pages': 23, 'return_df': True}
2286  pets added


In [ ]:
time.sleep(5)
cats_petpy_dict = create_petpy_dict(cat_params, cat_pages)
adoptable_cats = pf.animals(**cats_petpy_dict)
append_df(adoptable_cats, cats_petpy_dict)

In [19]:
assert(dog_pagination['total_count'] - 20 < adoptable_dogs.shape[0] < dog_pagination['total_count'] + 20)\
    ,f"API count not close to df size- df size: {adoptable_dogs.shape[0]}, api count: {dog_pagination['total_count']}"
assert(cat_pagination['total_count'] - 20 < adoptable_cats.shape[0] < cat_pagination['total_count'] + 20)\
    ,f"API count not close to df size- df size: {adoptable_cats.shape[0]}, api count: {cat_pagination['total_count']}"
